In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA
import lightgbm as lgb
import time

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id";
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [7]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [8]:
y = train['target'].values
X = feat_eng(train)
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [9]:
lgbm = lgb.Booster(model_file='lgbm_model.lgb')

In [10]:
lgbm_columns = ['stock_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'overall_medvol', 'first5min_medvol',
       'last5min_medvol', 'bid_plus_ask_sizes', 'imbalance_ratio', 'imb_s1',
       'imb_s2', 'ask_x_size', 'bid_x_size', 'ask_minus_bid',
       'bid_price_over_ask_price', 'reference_price_minus_far_price',
       'reference_price_times_far_price', 'reference_price_times_near_price',
       'reference_price_minus_ask_price', 'reference_price_times_ask_price',
       'reference_price_ask_price_imb', 'reference_price_minus_bid_price',
       'reference_price_times_bid_price', 'reference_price_bid_price_imb',
       'reference_price_minus_wap', 'reference_price_times_wap',
       'reference_price_wap_imb', 'far_price_minus_near_price',
       'far_price_times_near_price', 'far_price_minus_ask_price',
       'far_price_times_ask_price', 'far_price_minus_bid_price',
       'far_price_times_bid_price', 'far_price_times_wap', 'far_price_wap_imb',
       'near_price_minus_ask_price', 'near_price_times_ask_price',
       'near_price_ask_price_imb', 'near_price_minus_bid_price',
       'near_price_times_bid_price', 'near_price_bid_price_imb',
       'near_price_minus_wap', 'near_price_wap_imb',
       'ask_price_minus_bid_price', 'ask_price_times_bid_price',
       'ask_price_minus_wap', 'ask_price_times_wap', 'ask_price_wap_imb',
       'bid_price_minus_wap', 'bid_price_times_wap', 'bid_price_wap_imb',
       'reference_price_far_price_near_price_imb2',
       'reference_price_far_price_ask_price_imb2',
       'reference_price_far_price_bid_price_imb2',
       'reference_price_far_price_wap_imb2',
       'reference_price_near_price_ask_price_imb2',
       'reference_price_near_price_bid_price_imb2',
       'reference_price_near_price_wap_imb2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2',
       'far_price_near_price_ask_price_imb2',
       'far_price_near_price_bid_price_imb2', 'far_price_near_price_wap_imb2',
       'far_price_ask_price_bid_price_imb2', 'far_price_ask_price_wap_imb2',
       'far_price_bid_price_wap_imb2', 'near_price_ask_price_bid_price_imb2',
       'near_price_ask_price_wap_imb2', 'near_price_bid_price_wap_imb2',
       'ask_price_bid_price_wap_imb2', 'pca_prices']

In [11]:
lgbm_preds = lgbm.predict(X[lgbm_columns])

In [12]:
X['lgbm_preds'] = lgbm_preds
lgbm_preds =  []
del lgbm

In [13]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [01:21<14:23,  4.77s/it]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [08:01<08:31,  5.17s/it]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [10:21<04:58,  4.33s/it]

Missing Targets for day=35,for stock_id=131, Excluding


 79%|███████▉  | 158/200 [13:19<03:30,  5.02s/it]

Missing Targets for day=388,for stock_id=158, Excluding


 92%|█████████▏| 183/200 [15:25<01:18,  4.59s/it]

In [ ]:
for stock in trading_data.stocksDict.values():
    stock.data_daily = []

train = []

In [ ]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [ ]:
optim_dict = {
    'RMSProp':optim.RMSprop,
    "Adam":optim.Adam,
    "AdamW":optim.AdamW,
    'SGD':optim.SGD,
    
}

In [ ]:
def model_pipeline(trading_df=trading_data, config=None):
    trading_df = trading_data
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        config = wandb.config
        
        model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')
        config = wandb.config
        optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
        print(model)
        trading_df.reset_hidden(hidden_size=config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [ ]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [ ]:
trading_data.reset_hidden(16,num_layers=5)

In [ ]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]] 
hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [ ]:
hidden_in.shape

torch.Size([5, 191, 16])

In [ ]:
config_static = {'learning_rate':0.001, 'hidden_size':16, 'num_layers':2, 'batch_norm':1}

In [ ]:
config = config_static

In [ ]:
model = model_pipeline(trading_data, config_static)


GRUNetV2(
  (gru): GRU(81, 16, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(81, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): Identity()
  (fc0): Linear(in_features=3400, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/500 [00:00<?, ?it/s]

1t2-t1=0.04736609999963548
2t2-t1=1.0237489999999525
3t2-t1=0.039978500000870554
4t2-t1=0.0016562999990128446
5t2-t1=0.16893920000075013
1t2-t1=0.026640399999450892
2t2-t1=0.4354289000002609
3t2-t1=0.028669100000115577
4t2-t1=0.0007461999994120561
5t2-t1=0.14830290000099922
1t2-t1=0.02721409999867319
2t2-t1=0.28645550000146613
3t2-t1=0.026918600000499282
4t2-t1=0.0006895999977132306
5t2-t1=0.14156179999918095
1t2-t1=0.022585200000321493
2t2-t1=0.2830069999981788
3t2-t1=0.027065500002208864
4t2-t1=0.000698499999998603
5t2-t1=0.14105950000157463
1t2-t1=0.024324699999851873
2t2-t1=0.2763364999991609
3t2-t1=0.026012300000729738
4t2-t1=0.0009033000023919158
5t2-t1=0.1405440999988059
1t2-t1=0.021587400002317736
2t2-t1=0.2787739999985206
3t2-t1=0.026855299998715054
4t2-t1=0.0006768999992345925
5t2-t1=0.1422061000012036
1t2-t1=0.021629200000461424
2t2-t1=0.26102950000131386
3t2-t1=0.026296200001524994
4t2-t1=0.0007086999976309016
5t2-t1=0.14097319999928004
1t2-t1=0.021304000001691747
2t2-t1=0.

In [ ]:
model = torch_classes.GRUNetV2(81,config['hidden_size'],num_layers=config['num_layers'], batch_norm=config['batch_norm']).to('cuda:0')

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=4)

In [ ]:
len(targets)

201

In [ ]:
stock_targets = []
for i in range(0,200):
    print(i)
    try:
        stock_targets.append(torch.stack(trading_data.stocksDict[i].target_daily[4]))
    except Exception as e:
        print(e)
        stock_targets.append(torch.zeros(55))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
4
70
71
72
73
74
75
76
77
78
79
4
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
4
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
4
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
4
151
152
153
4
154
155
156
4
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
4


In [ ]:
len(stock_targets)

200

In [ ]:
stocks_hidden = [torch.stack(x) for x in stocks_hidden]
X = torch.cat(stocks_hidden,dim=-1)
Y = torch.stack(targets).transpose(0,1).to('cuda:0')

In [ ]:
output, relu = model(X)

In [ ]:
output.shape

torch.Size([55, 200])

In [ ]:
Y.shape

torch.Size([55, 200])

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "val_epoch_loss", "goal": "minimize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"values": ["adamW", 'adam', 'SGD', 'RMSprop']},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "num_layers": {"values": [2,3,4,5]},
    'hidden_size':{'values':[8,16,32,64,128,256,512,1024]},
    'learning_rate': {'max': 0.001, 'min': 0.00001},
    'batch_norm':{'values':[0,1,2]}
}

sweep_config["parameters"] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Optiver Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)